In [1]:
import os
import numpy as np
import scipy.io
import faiss
import plotly.graph_objects as go


In [7]:
import json

# Open the JSON file
with open('iitd_features_1_with seprate_left_abd_wrute.json', 'r') as file:
    data = json.load(file)  # Load JSON content into a Python dictionary or list

# print(len(data["X"][0]))
print(data["y"])



[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 13, 10013, 10013, 13, 10013, 13, 10013, 13, 13, 10013, 14, 10014, 14, 10014, 10014, 10014, 14, 14, 14, 10014, 15, 15, 10015, 10015, 15, 15, 10015, 10015, 10015, 15, 10016, 16, 16, 16, 10016, 10016, 16, 16, 10016, 10016, 10017, 10017, 10017, 10017, 17, 17, 17, 17, 10017, 17, 18, 18, 10018, 18, 10018, 10018, 18, 10018, 18, 10018, 19, 10019, 10019, 10019, 19, 19, 19, 10019, 10019, 19, 10020, 20, 10020, 20, 10020, 20, 20, 10020, 10020, 20, 21, 21, 10021, 21, 10021, 21, 10021, 21, 10021, 10021, 10022, 22, 22, 10022, 22, 10022, 22, 22, 10022, 10022, 10023, 10023, 23, 23, 10023, 

In [ ]:
import json
import numpy as np
import faiss
import plotly.graph_objects as go

# ---------------------
# Load JSON data
# ---------------------
def load_data(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)
    x = np.array(data["x"], dtype='float32')
    y = np.array(data["y"])
    return x, y

# ---------------------
# Split representatives & queries
# ---------------------
def split_representatives(x, y):
    unique_labels = np.unique(y)
    index_vectors = []
    index_labels = []
    query_vectors = []
    query_labels = []

    for label in unique_labels:
        indices = np.where(y == label)[0]
        if len(indices) == 0:
            continue
        # First sample as representative
        index_vectors.append(x[indices[0]])
        index_labels.append(label)
        # Remaining for queries
        if len(indices) > 1:
            remaining = indices[1:]
            query_vectors.extend(x[remaining])
            query_labels.extend(y[remaining])

    return (np.array(index_vectors, dtype='float32'),
            np.array(index_labels),
            np.array(query_vectors, dtype='float32'),
            np.array(query_labels))

# ---------------------
# Build HNSW Index
# ---------------------
def build_hnsw_index(vectors, dim, M=16, efConstruction=50):
    index = faiss.IndexHNSWFlat(dim, M)
    index.hnsw.efConstruction = efConstruction
    index.add(vectors)
    return index

# ---------------------
# Evaluate Hit Rate for varying efSearch
# ---------------------
def evaluate_hit_rate(index, index_labels, query_vectors, query_labels, ef_max=100):
    hit_rates = []
    ef_values = range(1, ef_max + 1)

    for ef in ef_values:
        index.hnsw.efSearch = ef
        distances, indices = index.search(query_vectors, k=1)

        # Get predicted labels
        predicted_labels = index_labels[indices.flatten()]

        # Calculate hits
        hits = np.sum(predicted_labels == query_labels)
        total = len(query_labels)
        hit_rate = hits / total if total > 0 else 0

        hit_rates.append(hit_rate)
        print(f"efSearch={ef}, Hit rate={hit_rate:.4f}")

    return ef_values, hit_rates

# ---------------------
# Plot Hit Rate vs efSearch (using Plotly)
# ---------------------
def plot_hit_rate_plotly(ef_values, hit_rates):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=list(ef_values),
        y=hit_rates,
        mode='lines+markers',
        name="Hit Rate",
        line=dict(width=2),
        marker=dict(size=6)
    ))
    fig.update_layout(
        title="HNSW Hit Rate vs efSearch",
        xaxis_title="efSearch",
        yaxis_title="Hit Rate",
        template="plotly_white",
        hovermode="x unified"
    )
    fig.show()

# ---------------------
# Main Workflow
# ---------------------
def main(json_path, ef_max=50):
    # Step 1: Load data
    x, y = load_data(json_path)

    # Step 2: Split representatives & queries
    index_x, index_y, query_x, query_y = split_representatives(x, y)

    # Step 3: Build HNSW index
    dim = index_x.shape[1]
    index = build_hnsw_index(index_x, dim)

    # Step 4: Evaluate hit rate
    ef_values, hit_rates = evaluate_hit_rate(index, index_y, query_x, query_y, ef_max=ef_max)

    # Step 5: Plot graph with Plotly
    plot_hit_rate_plotly(ef_values, hit_rates)

# Example usage
if __name__ == "__main__":
    main("iitd_features_1_with seprate_left_abd_wrute.json", ef_max=50)


FileNotFoundError: [Errno 2] No such file or directory: 'data.json'